# Code Starting Point

In [ ]:
# Import all required function to generate streams, queries and views
from pyspark.sql import SparkSession
from IPython.display import display, clear_output
import time
from pyspark.sql import functions as F
from pyspark.sql.types import StructType,StringType, StructField, IntegerType, FloatType, BinaryType, LongType, DoubleType
from pyspark.sql.window import Window
import os

In [ ]:
# Initiate a local spark session
spark = SparkSession.builder \
        .appName('kafka') \
        .getOrCreate()

In [ ]:
# Create a function to generate the first stream (all data columns with no filtering)

def generate_stocktrades_stream(keep_stream = False):
    
    # Define the raw Spark Stream
    stream_df = spark \
      .readStream \
      .format("kafka") \
      .option("kafka.bootstrap.servers", "broker:29092") \
      .option("startingOffsets", "earliest") \
      .option("subscribe", "STOCKTRADES_JSON") \
      .load()
    
    # Convert to string types for JSON conversion
    string_stream_df = stream_df \
        .withColumn("key", stream_df["key"].cast(StringType())) \
        .withColumn('value', stream_df["value"].cast(StringType()))
    
    # Define the Schema for the end JSON format
    schema_stocktrades =  StructType([
        StructField("SIDE", StringType(),  True),
        StructField("QUANTITY", IntegerType(),  True),
        StructField("PRICE", IntegerType(),  True),
        StructField("SYMBOL", StringType(),  True),
        StructField("ACCOUNT", StringType(), True),
        StructField("USERID", StringType(), True)
])
    # Convert the string type to json format stream
    json_stream_df = string_stream_df\
    .withColumn("value", F.from_json("value", schema_stocktrades))
    stocktrades_stream_df = json_stream_df \
    .select( \
        F.col("key").alias("event_key"), \
        F.col("topic").alias("event_topic"), \
        F.col("timestamp").alias("event_timestamp"), \
        "value.side", \
        "value.quantity", \
        "value.price", \
        "value.symbol", \
        "value.account", \
        "value.userid"
    )
    
    # Export a queryable view od the stream
    
    if not keep_stream:
        return stocktrades_stream_df \
        .writeStream \
        .format("memory") \
        .queryName("stocktrades_view") \
        .start()
    else:
        return stocktrades_stream_df \
        .writeStream \
        .format("memory") \
        .queryName("stocktrades_view") \
        .start(), stocktrades_stream_df

In [ ]:
# Run function to generate raw stocktrades stream
stocktrades_stream = generate_stocktrades_stream()

In [ ]:
# Show results (limit 20) of stream
clear_output(wait=True)
display(spark.sql('SELECT * FROM stocktrades_view').show(20))
time.sleep(1)

In [ ]:
# Stop the stream as it is no longer used
stocktrades_stream.stop()

In [ ]:
# Create function to stream filtered streams from Kafka as streaming dataframes
def generate_side_stream(SIDE):
    
    # Create string for source
    source_stream = SIDE + "_TRADES"
    # Define the raw Spark Stream
    
    stream_df = spark \
      .readStream \
      .format("kafka") \
      .option("kafka.bootstrap.servers", "broker:29092") \
      .option("startingOffsets", "latest") \
      .option("subscribe", source_stream) \
      .load()
    
    # Convert to string types for JSON conversion
    string_stream_df = stream_df \
        .withColumn("key", stream_df["key"].cast(StringType())) \
        .withColumn('value', stream_df["value"].cast(StringType()))
    
    # Define the Schema for the end JSON format
    schema_stocktrades =  StructType([
        StructField("QUANTITY", IntegerType(),  True),
        StructField("PRICE", IntegerType(),  True),
        StructField("SYMBOL", StringType(),  True),
        StructField("ACCOUNT", StringType(), True),
        StructField("USERID", StringType(), True)
])
    # Convert the string type to json format stream
    json_stream_df = string_stream_df\
    .withColumn("value", F.from_json("value", schema_stocktrades))
    stocktrades_stream_df = json_stream_df \
    .select( \
        F.col("key").alias("event_key"), \
        F.col("topic").alias("event_topic"), \
        F.col("timestamp").alias("event_timestamp"), \
        "value.quantity", \
        "value.price", \
        "value.symbol", \
        "value.account", \
        "value.userid"
    )
    
    # Export a queryable view od the stream
    return stocktrades_stream_df \
    .writeStream \
    .format("memory") \
    .queryName(SIDE + '_view') \
    .start()

In [ ]:
# Create views of the buy and sell streams
BUY_stream = generate_side_stream('BUY')
SELL_stream = generate_side_stream('SELL')

In [ ]:
# Show results (limit 20) of stream
clear_output(wait=True)
display(spark.sql('SELECT * FROM BUY_view').show(20))
time.sleep(1)

In [ ]:
# Show results (limit 20) of stream
clear_output(wait=True)
display(spark.sql('SELECT * FROM SELL_view').show(20))
time.sleep(1)

In [ ]:
# Stop the two streams
BUY_stream.stop()
SELL_stream.stop()

In [ ]:
### Create windowed dataframe

# Restart the raw data data frame
raw_stream, raw_stream_df = generate_stocktrades_stream(True)

# Create parameters for the window stream
window_duration = '60 seconds'
slide_duration = '30 seconds'

In [ ]:
# Create the windowed stream (this groups by symbol and then counts the numbers of trades in the window, quantity of shares traded and the average price traded)
windowed_agg_df = raw_stream_df \
    .withWatermark('event_timestamp', '1 minutes') \
    .groupBy(F.window(raw_stream_df.event_timestamp, window_duration, slide_duration), raw_stream_df.symbol) \
    .agg(F.count('SYMBOL').alias('no_trades'), \
    F.sum('QUANTITY').alias('tot_quantity'), \
    F.avg('PRICE').alias('avg_price'))

In [ ]:
# Export the stream as a qeuryable view
windowed_agg_stream = windowed_agg_df \
                        .writeStream \
                        .format("memory") \
                        .outputMode("Complete") \
                        .queryName("windowed_view") \
                        .start()

In [ ]:
# Show output from view
clear_output(wait=True)
display(spark.sql('SELECT * FROM windowed_view').show(20, False))
time.sleep(1)

In [ ]:
focused_df = raw_stream_df \
    .select('quantity', 'price', 'side', 'event_timestamp') \
    .filter(windowed_agg_df.symbol == 'ZVV')

In [ ]:
test_stream = focused_df \
                        .writeStream \
                        .format("memory") \
                        .outputMode("Complete") \
                        .queryName("test_stream") \
                        .start()

In [ ]:
# Show output from view
clear_output(wait=True)
display(spark.sql('SELECT * FROM test_stream').show(20, False))
time.sleep(1)

In [ ]:
# Write all available data to stream (parquet sink)
raw_stream_df.writeStream \
                .format("parquet") \
                .option("header", True) \
                .option("path", "parquet_output") \
                .option("checkpointLocation", "checkpoint/data") \
                .outputMode("append") \
                .trigger(once=True) \
                .start()

In [ ]:
# Stop the raw stream
raw_stream.stop()

In [ ]:
# Stop the windowed stream
windowed_agg_stream.stop()

In [ ]:
# Stop the spark instance
spark.stop()